In [ ]:
# check python version 
from platform import python_version

print(python_version())

In [ ]:
# check tensorflow version
import tensorflow as tf
print(tf.__version__)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
import math
import keras as k
import pandas as pd
import numpy as np
from copy import copy
import scipy.io
from sklearn.decomposition import PCA

from scipy.signal import find_peaks
from scipy import interpolate
from scipy.special import iv
from numpy import sin,cos,pi,array,linspace,cumsum,asarray,dot,ones
from pylab import plot, legend, axis, show, randint, randn, std,lstsq

from sklearn.manifold import MDS
import csv
import os
from tqdm import tqdm 

In [ ]:
%cd /home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1/Python_Inputs

# Ensure fourierseries.py is in the pathway
!ls -l fourierseries.py

In [ ]:
import util
import phaser
import dataloader
# Preprocess data for a single subject - to be send to modeling frameworks
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k

In [ ]:
def vonMies(t,t_0, b):
    out = np.exp(b*np.cos(t-t_0))/(2*pi*iv(0, b))
    return out

In [ ]:
# The path to save the models and read data from
path = '/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1'

# Insert the directory
import sys
sys.path.insert(0,path)

In [ ]:
# Non-changing variables 

# number of trials in dataset 
trialnum = 252 # 72 total trials

# number of samples in each trial
trialsamp = 2100

# number of features collected per trial
feats = 15

#Batch size - same as the number of traintrials
batch_size = trialnum

# Number of Layers
numlayers = 1

# Choose the number of iterations to train the model- if this script has been run previously enter a value greater than was 
# inputted before and rerun the script. 
finalepoch = 10000

# load the input data/kinematics
datafilepath = '/home/victorphilippecodex/Documents/Hand_Signature/WholeGroup1/DownsampledGroup1.csv' #input data
all_csvnp = np.loadtxt(datafilepath,delimiter=',').T

# reshape all the input data into a tensor
all_inputdata_s = all_csvnp.reshape(trialnum,trialsamp,feats) 
csvnp = all_inputdata_s
print('original input data shape is: '+ str(all_csvnp.shape ))
print('input data reshaped is: '+ str(all_inputdata_s.shape))

In [ ]:
# generate a list of models and corresponding parameters to test 
test_model_nodes = [256] 
seqs = [699] #lookback parameter

# run multiple model architechtures many times to test stability of cost function outputs
runs = 1 # stability analysis - repeat each model architecture this many times
test_model_seq = np.repeat(seqs, runs)

count = np.arange(runs)

All_nodes = np.empty([0,1], dtype='int')
All_seq = np.empty([0,1],dtype='int')
All_valseg = np.empty([0,1],dtype='int')
All_trainseg = np.empty([0,1],dtype='int')
All_modelname = []
All_mod_name = []
count = np.empty([0,1],dtype='int'); #initialize model run -- this serves as the model run ID number
ct = 0
for a in test_model_nodes:
  for b in test_model_seq:
    count = np.append(count,  ct + 1 )
    #if statement for valseg, trainseg based on sequence length
    if int(b) == 699:
      trainseg = 2
      valseg = 1
    

    All_nodes = np.append(All_nodes, a) 
    All_seq = np.append(All_seq, int(b))
    All_valseg = np.append(All_valseg, valseg)
    All_trainseg = np.append(All_trainseg, trainseg)
    All_modelname = np.append(All_modelname, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) + '/' )
    All_mod_name = np.append(All_mod_name, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) )

    if ct+1 < runs:
      ct += 1
    else:
      ct = 0

print(All_mod_name)
j = 0
print(path + All_modelname[j] + All_mod_name[j] + '_bestwhole.h5') 

In [ ]:
for j in range(len(All_mod_name)):
    # make folder to store model
    newfoldpath = path + All_mod_name[j]


    # extract path to store each model and generated data
    savepath = path + All_modelname[j]
    mod_name = All_mod_name[j]

    print('Working on: ' + mod_name + ' model ' + str(j) + ' / ' + str(len(All_mod_name)))
    
    # Specify variables for model run instance

    # Number of Units
    numunits = All_nodes[j]
    
    test_ind = np.arange(0, len(all_inputdata_s), 1)  # run all input data through

    ext_drive= trialsamp

    tot_len = len(test_ind)

    # PCA of HCs

    ExternalDriveHCs_testset = np.load(savepath + mod_name + '_extdriveHCs_testset.npy')
    SelfDriveHCs_testset = np.load(savepath + mod_name + '_selfdriveHCs.npy')

    X = ExternalDriveHCs_testset #PCA of only externally driven

    pca = PCA(n_components=numunits*2) #initialize

    X_reduction = pca.fit_transform(X) # X is projected on the first principal components previously extracted from a training set - data is centered here

    np.save(savepath + mod_name + '_pca_var.npy', pca.explained_variance_ratio_[0:6])

    tot_len = len(test_ind) # len of all trials
    HC_CellArray = np.empty(shape=[tot_len, ext_drive-100, X_reduction.shape[1]])
    start = 0
    last = ext_drive # length of trials
    for p in range(tot_len):
        HC_CellArray[p]= X_reduction[start+100:last]
        start = start+ext_drive
        last = last+ext_drive

    scipy.io.savemat(savepath + mod_name + '_HCvalues.mat', {'HC_CellArray': HC_CellArray})
    trialsamp = trialsamp- 100
    # Generate phase averaged signatures
    print('generating phase averaged signatures')
    # phase length = 100
    lim = tot_len
    L = HC_CellArray.shape[2] # lets look at 1st 6 to speed things up #numunits*2  # number of PCs or units to be phase averaged
    PhaseAveragedPCs = np.empty(shape=[lim, L, 100])
    PhaseAveragedPCs_shift = np.empty(shape=[lim, L, 100])
    Phase_Variables = np.empty(shape=[lim, trialsamp])
    cyclephase = []
    PC_Shifts = np.empty(shape=[lim]) # store phase shift values
    # Average orbits initialization
    numSegments = 100 # we want each Phase averaged orbit to be 100 sample points long
    phaseVals = np.linspace(0, 2*pi, numSegments, endpoint=True) # phases that we want our phase averaged orbits to correspond to
    kappa = 20 

    for a in range(lim): #for length of all the trials 
        print(['processing trial: ' + str(a)]) 
        dats=[] # reset variables after each trial - kinematics
        dats2 = [] # reset the HC params
        allsigs = np.empty(shape=[100,])
        all_phase_var = np.empty(shape=[100,]) 
        PhaseAvgPCs = np.empty(shape=[100])
        all_cyclephase = []
        raw = HC_CellArray[a][:,(0,0)].T # Only use 1st 3 PCs (of HCs),
        #raw = HC_CellArray[a][:,0:2].T
        #raw = raw[:,np.newaxis]
        raw2 = HC_CellArray[a][:,0:L].T  # extract all the HC data
        for b in range(6): #Shai's code does duplication- works better than without duplicating data
            dats.append(raw-raw.mean(axis=1)[:,np.newaxis]) #center the HCs data for phase estimation -- this centers each one separately
            dats2.append(raw2-raw2.mean(axis=1)[:,np.newaxis])
        #dats = np.swapaxes(dats,1,2)
        rHC = dats2[0] #centered
        #rHC = raw2 #uncentered 
        phr = phaser.Phaser(dats) # use centered data from 1st 3 PCs
        phi = [ phr.phaserEval( d ) for d in dats ] # extract phase
        phi2  =(phi[0].T % (2*pi)); # Take modulo s.t. phase ranges from [0,2*pi]
         
        # find average orbits
        avgOrbits = np.zeros((numSegments,L)) #initialize avg orbits
        phases = np.reshape(phi2,[trialsamp,]) #this was the error!!!!!

        for c in range(numSegments): #number of points in final average orbit/phase averaging
            vonMiesCurrent = vonMies(phases,phaseVals[c],kappa) # for each value in the num of phase points calculate current
            sumVal = np.sum(vonMiesCurrent)

            for d in range(L):  # for the number of features or units
                data = np.reshape(rHC[d,:],[trialsamp,1])
                avgOrbits[c,d] = np.sum(data.T*vonMiesCurrent)/sumVal # phase point (row), feature of PC (column) -- this is generating a value for the 1st phase point of each feature

        PhaseAveragedPCs[a] = avgOrbits.T # transform to match overall saving structure as before
        phi2 = phi2.reshape(trialsamp,)
        Phase_Variables[a] = phi2 # store phase variables for all cycles/features - may need to phase shift these 
        
        # Phase shift the data according to PC1 max align with zero phase
        PC1_maxloc = np.argmax(PhaseAveragedPCs[a][0]) # only PC1 value (1st max if repeated)
        Data2Shift = PhaseAveragedPCs[a]
        NewP = np.roll(Data2Shift, -PC1_maxloc,axis=1) #shift back to orgin
        PC_Shifts[a] = PC1_maxloc
        PhaseAveragedPCs_shift[a] = NewP

    np.save(savepath + mod_name + '_PhaseAvgPcs_shift.npy', PhaseAveragedPCs_shift)
    scipy.io.savemat(savepath + mod_name + '_PhaseAvgPcs_shift.mat', {'PhaseAvgPCs_shift': PhaseAveragedPCs_shift})

    np.save(savepath + mod_name + '_PhaseAvgPcs.npy', PhaseAveragedPCs)
    scipy.io.savemat(savepath + mod_name + '_PhaseAvgPcs.mat', {'PhaseAvgPCs': PhaseAveragedPCs})

    np.save(savepath + mod_name + '_PhaseVariables.npy', Phase_Variables)
    scipy.io.savemat(savepath + mod_name + '_PhaseVariables.mat', {'PhaseVariables': Phase_Variables})


    # Concatenate gait signature per trial
    # Make a single phase averaged signature per row
    PA_shape = PhaseAveragedPCs_shift.shape
    gait_sig_size = PA_shape[1]*PA_shape[2]
    Gaitsignatures = np.empty(shape = [len(PhaseAveragedPCs_shift),gait_sig_size]) #change to shifted PCS! 
    Gaitsignatures_trunc6 = np.empty(shape = [len(PhaseAveragedPCs_shift),600])
    for h in range(len(PhaseAveragedPCs_shift)):
        reshape_sig = PhaseAveragedPCs_shift[h].reshape(1,gait_sig_size)
        Gaitsignatures[h] = reshape_sig[0]
        Gaitsignatures_trunc6[h] = reshape_sig[0][0:600] #truncate to 1st 6 PCs
    np.save(savepath + mod_name + '_Gaitsignatures.npy', Gaitsignatures)
    scipy.io.savemat(savepath + mod_name + '_Gaitsignatures.mat', {'GaitSigs': Gaitsignatures})

    np.save(savepath + mod_name + '_Gaitsignatures_trunc6.npy', Gaitsignatures_trunc6)
    scipy.io.savemat(savepath + mod_name + '_Gaitsignatures_trunc6.mat', {'GaitSigs': Gaitsignatures_trunc6})
    

In [ ]:
plt.plot(Gaitsignatures_trunc6[0].T)
plt.plot(Gaitsignatures_trunc6[27].T)

In [ ]:
    
#datafilepath3 = '/home/victorphilippecodex/Downloads/SpeedLabelsPatrick.mat'
#speedlabels = scipy.io.loadmat(datafilepath3)

#speedlist = []
#for m in test_ind:
    #speedlist.append(speedlabels['FingTrials'][m][0][0][0])

#myfile = '/home/victorphilippecodex/Downloads/Subjectlabels .csv'
#csvfile = open(myfile, 'r')
#reader = csv.reader(csvfile, delimiter='\t')
#allsubs = list(reader)
#select_subs = map(allsubs.__getitem__, test_ind)
#subs= list(select_subs)


# Mutlidimensional scaling of gait signatures
embedding = MDS(n_components=3)
X_transformed = embedding.fit_transform(Gaitsignatures_trunc6)
np.save(savepath + mod_name + '_MDS_X_transformed.npy', X_transformed)
scipy.io.savemat(savepath + mod_name + '_MDS_X_transformed.mat', {'X_transformed': X_transformed})


# MDS plots of ext and self sigs
#fig = plt.figure()
#plt.scatter(X_transformed[:,0],X_transformed[:,1],c = speedlist)
# Loop for annotation of all points
#for i in range(len(subs)):
        
    #plt.annotate(str(count), (X_transformed[i,0], X_transformed[i,1] + 0.3))
    #count = count + 1    
#plt.savefig(savepath + mod_name +  '_MDS.png', dpi = 300)
#plt.close(fig)# close figure in loop

In [ ]:
plt.plot(HC_CellArray[3,:,0:3])

In [ ]:
tot_len

In [ ]:
plt.plot(HC_CellArray[91,:,0])

In [ ]:
plt.plot(HC_CellArray[14,:,0:2])

In [ ]:
np.load("/home/victorphilippecodex/Documents/Hand_Signature/Individual0011UNIT_256_LB_4999_run_1/UNIT_256_LB_4999_run_1_pca_var.npy")